<a href="https://colab.research.google.com/github/simonbustamante/mit-modelospredictivos/blob/master/Caso_de_Estudio_6_2_Dise%C3%B1o_de_predicciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caso de Estudio 6.2 - Diseño de predicciones con datos sobre ventas en el Reino Unido

# Configuración

Ejecute (Run) estas celdas para instalar los paquetes necesarios para completar el caso de estudio. Esto podría llevar unos minutos así que sea paciente.

<h1 style="color:red;">ATENCIÓN: Puede que vea errores cuando ejecute las celdas siguientes. Sin embargo, no se preocupe, con tal de que pueda ejecutar la celda de importación de librerías (dos más abajo) y vea el mensaje "Librerías importadas con éxito!" es suficiente y puede continuar con el caso de estudio.<h1>

In [ ]:
!pip install -q featuretools==1.29.0
import featuretools as ft
print('Librerías instaladas con éxito!')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

<h1>Atención:</h1>

Ahora ha de reiniciar el entorno de ejecución. Para ello vaya a:

> Entorno de ejecución > _Reiniciar entorno de ejecución_

en la parte superior de su pantalla. Esto asegurará que sus cambio se han realizado con éxito.


# Importar

Sincronice su cuenta de Google. Para ello, siga el link que aparece en la salida de la siguiente celda una vez ejecutada. Copie el código que le aparece en pantalla e introdúzcalo en la salida de la celda. Una vez vea el mensaje: `Google Drive sincronizado con éxito!` puede continuar ejecutando el resto de celdas.

In [ ]:
!mkdir uk-retail-data

In [ ]:
from google.colab import auth
auth.authenticate_user()

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

data = drive.CreateFile({'id':'1feQJdQqOYUaxeUcC7fj7s-ib3Wuqj4Hf'})
data.GetContentFile('uk-retail-data/customers.csv')
data = drive.CreateFile({'id':'16spEigNWUceuyB5uyaFyo5gW3YX5Fkko'})
data.GetContentFile('uk-retail-data/invoices.csv')
data = drive.CreateFile({'id':'1RlgJaKznx931rnp-vUpki_zHOtkq1eEy'})
data.GetContentFile('uk-retail-data/item_purchases.csv')
data = drive.CreateFile({'id':'1Iwf8zTmLzZyyuhJg3CLc_zW4ZtpoviAy'})
data.GetContentFile('uk-retail-data/items.csv')

utils_file = drive.CreateFile({'id':'1UbV2z7L5vonCz3KFLywGs4U-p2g5gQwy'})
utils_file.GetContentFile('utils.py')

print('Google Drive sincronizado con éxito!')

Importe las librerías necesarias para el desarrollo del caso.

In [ ]:
import featuretools as ft
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import utils

from utils import (find_training_examples,
                   load_uk_retail_data,
                   engineer_features_uk_retail,
                   preview,
                   feature_importances)

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

print('Librerías importadas con éxito!')

# Datos

Cargue la base de datos de ventas de diferentes productos en Reino Unido. Esto podría llevar unos minutos, así que sea paciente.

In [ ]:
item_purchases, invoices, items, customers = load_uk_retail_data()
preview(items, 10)

In [ ]:
item_purchases = item_purchases.dropna(subset="item_purchase_id")
items = items.dropna(subset="StockCode")
customers = customers.dropna(subset="CustomerID")
invoices = invoices.dropna(subset="InvoiceNo")

In [ ]:
entities = {"item_purchases": (item_purchases,
                               "item_purchase_id",
                               "InvoiceDate"),
            "items": (items,
                      "StockCode"),
            "customers": (customers,
                          "CustomerID"),
            "invoices":(invoices,
                        "InvoiceNo",
                        "first_item_purchases_time")
            }
entities

In [ ]:
relationships = [("customers","CustomerID","invoices","CustomerID"),
                 ("invoices", "InvoiceNo","item_purchases", "InvoiceNo"),
                 ("items", "StockCode","item_purchases", "StockCode")]

## Encontrando ejemplos de entrenamiento

In [ ]:
label_times = pd.DataFrame()

In [ ]:
label_times = find_training_examples(item_purchases,
                                     invoices,
                                     prediction_window = pd.Timedelta("14d"),
                                     training_window = pd.Timedelta("21d"),
                                     lead = pd.Timedelta("7d"),
                                     threshold = 5)


In [ ]:
label_times = label_times[['CustomerID','cutoff_time', 'purchases>threshold']].rename(columns={'cutoff_time': 'time', 'purchases>threshold': 'label'})

In [ ]:
preview(label_times, 5)

## Generación de variables

In [ ]:
from featuretools.primitives import (Count, Day, Hour, Max, Mean, Median, Min,
                                     Minute, Month, Std, Sum, Week, Weekday,
                                     IsWeekend)

trans_primitives = [Minute, Hour, Day, Week, Month, Weekday, IsWeekend]
agg_primitives = [Mean, Max, Std]
training_window='21d'

feature_matrix, features = ft.dfs(dataframes=entities,
                                  relationships=relationships,
                                  target_dataframe_name="customers",
                                  trans_primitives=trans_primitives,
                                  agg_primitives=agg_primitives,
                                  cutoff_time=label_times,
                                  cutoff_time_in_index=True,
                                  training_window=training_window)
feature_matrix.drop("Country", axis=1, inplace=True)
feature_matrix = feature_matrix.sort_index()

In [ ]:
preview(feature_matrix, 10)

## Entrenamiento de un modelo con *Random Forest*

In [ ]:
X_y = feature_matrix
y = X_y.pop('label')
X_train, X_test, y_train, y_test = train_test_split(feature_matrix,
                                                    y,
                                                    test_size=0.35)

In [ ]:
imp = SimpleImputer(missing_values=np.nan,
                    strategy='mean')
imp = imp.fit(X_train)
X_train_imp = imp.transform(X_train)

In [ ]:
clf = RandomForestClassifier(random_state=0,
                             n_estimators=100,
                             class_weight="balanced",
                             verbose=True)
clf.fit(X_train_imp, y_train)

## Prueba del modelo

In [ ]:
X_test_imp = imp.transform(X_test)
predicted_labels = clf.predict(X_test_imp)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, predicted_labels).ravel()
tn, fp, fn, tp

In [ ]:
feature_importances(clf,feature_matrix.columns, n=15)